# __Econometric Game__: Effect of electoral accountability on corruption?
   
### Team 3: Katheryn Ding, Amber Wei, Max Ye



## Set up 

In [92]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.special import expit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")


In [93]:
corruption_df = pd.read_stata("corruptiondata.dta")
corruption_df.columns

Index(['uf', 'nsorteio', 'totrecursos', 'tot_os', 'pop', 'purb',
       'p_secundario', 'cod_ibge6', 'pib_capita_02', 'op_01_04',
       ...
       'uf_d18', 'uf_d19', 'uf_d20', 'uf_d21', 'uf_d22', 'uf_d23', 'uf_d24',
       'uf_d25', 'uf_d26', 'esample2'],
      dtype='object', length=116)

In [94]:
# Define covariates for each category

# Mayor characteristics
mayor_covariates = [
    "pref_idade_tse",  # Age
    "pref_masc",       # Gender
    "pref_escola",     # Schooling
    "winmargin2000",   # Margin of victory in 2000
    "exp_prefeito"     # Was previously a mayor in a consecutive term
] + [col for col in corruption_df.columns if col.startswith("party_d")]

# Municipal characteristics
municipal_covariates = [
    "lpop",           # Log of population in 2000
    "purb",           # Percentage of population in urban sectors
    "p_secundario",   # Percentage with at least secondary education
    "mun_novo",       # New municipality indicator
    "lpib02",         # Log of GDP per capita in 2002
    "gini_ipea"       # Gini coefficient
]

# Political and Judicial characteristics
political_judicial_covariates = [
    "ENEP2000",  # Effective number of parties in 2000 mayor elections
    "ENLP2000",  # Effective number of parties in 2000 legislative elections
    "p_cad_pref" # Proportion of legislators from the same party as the mayor
]

# Dummies
dummy_covariates = [
    col for col in corruption_df.columns if col.startswith("uf_d") or col.startswith("sorteio")
]


print("Mayor Covariates:", mayor_covariates)
print("Municipal Covariates:", municipal_covariates)
print("Political and Judicial Covariates:", political_judicial_covariates)
print("Dummy Covariates:", dummy_covariates)

Mayor Covariates: ['pref_idade_tse', 'pref_masc', 'pref_escola', 'winmargin2000', 'exp_prefeito', 'party_d1', 'party_d3', 'party_d4', 'party_d5', 'party_d6', 'party_d7', 'party_d8', 'party_d9', 'party_d10', 'party_d11', 'party_d12', 'party_d13', 'party_d14', 'party_d15', 'party_d16', 'party_d17', 'party_d18']
Municipal Covariates: ['lpop', 'purb', 'p_secundario', 'mun_novo', 'lpib02', 'gini_ipea']
Political and Judicial Covariates: ['ENEP2000', 'ENLP2000', 'p_cad_pref']
Dummy Covariates: ['sorteio1', 'sorteio2', 'sorteio3', 'sorteio4', 'sorteio5', 'sorteio6', 'sorteio7', 'sorteio8', 'sorteio9', 'sorteio10', 'uf_d1', 'uf_d2', 'uf_d3', 'uf_d4', 'uf_d5', 'uf_d6', 'uf_d7', 'uf_d8', 'uf_d9', 'uf_d10', 'uf_d11', 'uf_d12', 'uf_d13', 'uf_d14', 'uf_d15', 'uf_d16', 'uf_d17', 'uf_d18', 'uf_d19', 'uf_d20', 'uf_d21', 'uf_d22', 'uf_d23', 'uf_d24', 'uf_d25', 'uf_d26']


In [95]:
corruption_df["log_valor_corrupt"] = np.log(corruption_df["valor_corrupt"] + 1)

# Define the treatment and outcome variables
treatment = "first"  
outcomes = ["pcorrupt", "ncorrupt_os", "valor_corrupt",'log_valor_corrupt'] 

# Use all covariates
all_covariates = (
    mayor_covariates +
    municipal_covariates +
    political_judicial_covariates +
    dummy_covariates
)
#set up dataset:
required_columns = [treatment] + outcomes + all_covariates
double_ml_dataset = corruption_df[required_columns].dropna()

print("Dataset Columns:", double_ml_dataset.columns)
print("Number of rows in the Dataset:", double_ml_dataset.shape[0])

Dataset Columns: Index(['first', 'pcorrupt', 'ncorrupt_os', 'valor_corrupt',
       'log_valor_corrupt', 'pref_idade_tse', 'pref_masc', 'pref_escola',
       'winmargin2000', 'exp_prefeito', 'party_d1', 'party_d3', 'party_d4',
       'party_d5', 'party_d6', 'party_d7', 'party_d8', 'party_d9', 'party_d10',
       'party_d11', 'party_d12', 'party_d13', 'party_d14', 'party_d15',
       'party_d16', 'party_d17', 'party_d18', 'lpop', 'purb', 'p_secundario',
       'mun_novo', 'lpib02', 'gini_ipea', 'ENEP2000', 'ENLP2000', 'p_cad_pref',
       'sorteio1', 'sorteio2', 'sorteio3', 'sorteio4', 'sorteio5', 'sorteio6',
       'sorteio7', 'sorteio8', 'sorteio9', 'sorteio10', 'uf_d1', 'uf_d2',
       'uf_d3', 'uf_d4', 'uf_d5', 'uf_d6', 'uf_d7', 'uf_d8', 'uf_d9', 'uf_d10',
       'uf_d11', 'uf_d12', 'uf_d13', 'uf_d14', 'uf_d15', 'uf_d16', 'uf_d17',
       'uf_d18', 'uf_d19', 'uf_d20', 'uf_d21', 'uf_d22', 'uf_d23', 'uf_d24',
       'uf_d25', 'uf_d26'],
      dtype='object')
Number of rows in the Data

## Lasso 

#### Assuming Heterogeneosity 

In [96]:

theta_hat_dr_values = []
mse_values = []

# Loop through each outcome
for outcome in outcomes:
    
    X = double_ml_dataset[all_covariates]
    D = double_ml_dataset[treatment]
    Y = double_ml_dataset[outcome]
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Prepare 5-Fold Cross-Fitting
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    dr_ate_fold = []
    mse_fold = []
    
    for train_index, test_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        D_train, D_test = D.iloc[train_index], D.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
        
        # Estimate propensity scores using Lasso
        lasso_pscore = LassoCV(alphas=np.logspace(-4, 0, 50), cv=5, random_state=42).fit(X_train, D_train)
        D_pred = lasso_pscore.predict(X_test)
        propensity_scores = expit(D_pred) 
        
        # Trim p-scores (0.01, 0.99)
        trimmed_indices = (propensity_scores > 0.01) & (propensity_scores < 0.99)
        trimmed_X = X_test[trimmed_indices]
        trimmed_D = D_test.iloc[trimmed_indices]
        trimmed_Y = Y_test.iloc[trimmed_indices]
        trimmed_pscore = propensity_scores[trimmed_indices]
        
        # Fit outcome models for treated (D=1) and untreated (D=0) groups
        # Treated model
        treated_model = LassoCV(alphas=np.logspace(-4, 0, 50), cv=5, random_state=42)
        treated_model.fit(X_train[D_train == 1], Y_train[D_train == 1])
        gamma1 = treated_model.predict(trimmed_X)
        
        # Untreated model
        untreated_model = LassoCV(alphas=np.logspace(-4, 0, 50), cv=5, random_state=42)
        untreated_model.fit(X_train[D_train == 0], Y_train[D_train == 0])
        gamma0 = untreated_model.predict(trimmed_X)
        
        # Step 3: Calculate doubly robust estimates for potential outcomes
        trimmed_data = pd.DataFrame({
            "gamma1": gamma1,
            "gamma0": gamma0,
            "D": trimmed_D.values,
            "Y": trimmed_Y.values,
            "pscore": trimmed_pscore,
        })
        
        # DR Estimate for Y(1)
        trimmed_data['Y1_dr'] = (
            trimmed_data['gamma1'] +
            (trimmed_data['D'] / trimmed_data['pscore']) * (trimmed_data['Y'] - trimmed_data['gamma1'])
        )
        
        # DR Estimate for Y(0)
        trimmed_data['Y0_dr'] = (
            trimmed_data['gamma0'] +
            ((1 - trimmed_data['D']) / (1 - trimmed_data['pscore'])) * (trimmed_data['Y'] - trimmed_data['gamma0'])
        )
        
        # Calculate treatment effect for the fold
        dr_ate_fold.append(np.mean(trimmed_data['Y1_dr'] - trimmed_data['Y0_dr']))
        
        # Calculate MSE for the fold
        mse_fold.append(mean_squared_error(
        trimmed_data['Y'], 
        trimmed_data['Y1_dr'] * trimmed_data['D'] + trimmed_data['Y0_dr'] * (1 - trimmed_data['D'])
        ))
    
    # Average treatment effect and MSE across folds
    dr_ate = np.mean(dr_ate_fold)
    mse = np.mean(mse_fold)
    
    # Store results
    theta_hat_dr_values.append(dr_ate)
    mse_values.append(mse)

# Create results DataFrame
ls_result_df_hetero = pd.DataFrame({
    "Outcome": outcomes,
    "Estimate": theta_hat_dr_values,
    "MSE": mse_values
})

# Print the final results DataFrame
print("Doubly Robust Treatment Effect Estimates with Cross-Fitting:")
print(ls_result_df_hetero)

Doubly Robust Treatment Effect Estimates with Cross-Fitting:
             Outcome       Estimate           MSE
0           pcorrupt      -0.022035  1.736679e-02
1        ncorrupt_os      -0.008515  3.261818e-03
2      valor_corrupt -148571.437500  1.017620e+12
3  log_valor_corrupt      -1.067475  3.614077e+01


In [97]:
# Checking for balance
weights_treated = 1 / propensity_scores[trimmed_D == 1]
weights_untreated = 1 / (1 - propensity_scores[trimmed_D == 0])

# Create DataFrames for treated and untreated groups
treated_data = trimmed_X[trimmed_D == 1]
untreated_data = trimmed_X[trimmed_D == 0]

# Compute weighted means for each covariate
treated_means = np.average(treated_data, axis=0, weights=weights_treated)
untreated_means = np.average(untreated_data, axis=0, weights=weights_untreated)

# Check absolute standardized mean differences
smd = np.abs(treated_means - untreated_means) / np.std(trimmed_X, axis=0)
print("Standardized Mean Differences (SMD):", smd)

Standardized Mean Differences (SMD): [0.06885722 0.2901101  0.02904402 0.6848417  0.21398759 0.06088972
 0.25428903 0.41079307 0.09445282 0.04607635 0.6        0.6
 0.35854152 0.24288547 0.24353866 0.03111833 0.16987991 0.2
 0.17568141 0.1651254  0.21418196 0.6        0.02996126 0.04102391
 0.18319435 0.20693757 0.4547287  0.39264885 0.3478793  0.23060311
 0.2800635  0.00521759 0.20551337 0.15774368 0.38865724 0.08364097
 0.05980109 0.2552489  0.47479638 0.35756427 0.02450931 0.18689632
 0.19421004 0.06287777 0.35430452 0.26188627 0.01593611 0.25
 0.46131605 0.3687714  0.05878664 0.0249002  0.26350877 0.01907789
 0.1791523  0.16089885 0.17051342 0.3866724  0.17795302 0.25078654
 0.23934054 0.625      0.04331093 0.07217446 0.17713386 0.03650402
 0.4       ]


Not very balanced.

#### Assuming Homogeneous treatment effct 

In [98]:
theta_hat_dr_values = []
mse_values = []

# Loop through each outcome
for outcome in outcomes:
    
    X = double_ml_dataset[all_covariates]
    D = double_ml_dataset[treatment]
    Y = double_ml_dataset[outcome]
    
    # Standardize the covariates
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Prepare 5-Fold Cross-Fitting
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    dr_ate_fold = []
    mse_fold = []
    
    for train_index, test_index in kf.split(X_scaled):
        # Split into training and testing sets
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        D_train, D_test = D.iloc[train_index], D.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
        
        # Step 1: Estimate propensity scores using Lasso
        lasso_pscore = LassoCV(alphas=np.logspace(-4, 0, 50), cv=5, random_state=42).fit(X_train, D_train)
        D_pred = lasso_pscore.predict(X_test)
        propensity_scores = expit(D_pred)  
        
        # Step 2: Trim p-scores (0.01, 0.99)
        trimmed_indices = (propensity_scores > 0.01) & (propensity_scores < 0.99)
        trimmed_X = X_test[trimmed_indices]
        trimmed_D = D_test.iloc[trimmed_indices]
        trimmed_Y = Y_test.iloc[trimmed_indices]
        trimmed_pscore = propensity_scores[trimmed_indices]
        
        # Step 3: Fit a single outcome model for homogeneous treatment effect
        outcome_model = LassoCV(alphas=np.logspace(-4, 0, 50), cv=5, random_state=42)
        outcome_model.fit(X_train, Y_train)
        gamma = outcome_model.predict(trimmed_X)
        
        # Step 4: Calculate doubly robust estimates for potential outcomes
        trimmed_data = pd.DataFrame({
            "gamma": gamma,
            "D": trimmed_D.values,
            "Y": trimmed_Y.values,
            "pscore": trimmed_pscore,
        })
        
        # DR Estimate for Y(1)
        trimmed_data['Y1_dr'] = (
            trimmed_data['gamma'] +
            (trimmed_data['D'] / trimmed_data['pscore']) * (trimmed_data['Y'] - trimmed_data['gamma'])
        )
        
        # DR Estimate for Y(0)
        trimmed_data['Y0_dr'] = (
            trimmed_data['gamma'] +
            ((1 - trimmed_data['D']) / (1 - trimmed_data['pscore'])) * (trimmed_data['Y'] - trimmed_data['gamma'])
        )
        
        # Step 5: Calculate treatment effect and MSE for the fold
        dr_ate_fold.append(np.mean(trimmed_data['Y1_dr'] - trimmed_data['Y0_dr']))
        mse_fold.append(mean_squared_error(
            trimmed_data['Y'], 
            trimmed_data['Y1_dr'] * trimmed_data['D'] + trimmed_data['Y0_dr'] * (1 - trimmed_data['D'])
        ))
    
    # Average treatment effect and MSE across folds
    dr_ate = np.mean(dr_ate_fold)
    mse = np.mean(mse_fold)
    
    # Store results
    theta_hat_dr_values.append(dr_ate)
    mse_values.append(mse)

# Create results DataFrame
ls_result_df_homo = pd.DataFrame({
    "Outcome": outcomes,
    "Estimate": theta_hat_dr_values,
    "MSE": mse_values
})

# Display the results DataFrame
print("Doubly Robust Treatment Effect Estimates with Cross-Fitting (Homogeneous):")
print(ls_result_df_homo)

Doubly Robust Treatment Effect Estimates with Cross-Fitting (Homogeneous):
             Outcome       Estimate           MSE
0           pcorrupt      -0.022970  1.742515e-02
1        ncorrupt_os      -0.008828  3.101943e-03
2      valor_corrupt -133287.843750  7.374056e+11
3  log_valor_corrupt      -1.086619  3.406843e+01


## Random Forest

In [99]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

#### Assuming Heterogeneous Treatment Effect

In [100]:
theta_hat_dr_values = []
mse_values = []

# Loop through each outcome
for outcome in outcomes:
    
    X = double_ml_dataset[all_covariates]
    D = double_ml_dataset[treatment]
    Y = double_ml_dataset[outcome]
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Prepare 5-Fold Cross-Fitting
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    dr_ate_fold = []
    mse_fold = []
    
    for train_index, test_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        D_train, D_test = D.iloc[train_index], D.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
        
        # Estimate propensity scores using Random Forest
        rf_pscore = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
        rf_pscore.fit(X_train, D_train)
        propensity_scores = rf_pscore.predict_proba(X_test)[:, 1]  # Probability of treatment
        
        # Trim p-scores (0.01, 0.99)
        trimmed_indices = (propensity_scores > 0.01) & (propensity_scores < 0.99)
        trimmed_X = X_test[trimmed_indices]
        trimmed_D = D_test.iloc[trimmed_indices]
        trimmed_Y = Y_test.iloc[trimmed_indices]
        trimmed_pscore = propensity_scores[trimmed_indices]
        
        # Fit outcome models for treated (D=1) and untreated (D=0) groups using Random Forest
        treated_model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
        treated_model.fit(X_train[D_train == 1], Y_train[D_train == 1])
        gamma1 = treated_model.predict(trimmed_X)
        
        untreated_model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
        untreated_model.fit(X_train[D_train == 0], Y_train[D_train == 0])
        gamma0 = untreated_model.predict(trimmed_X)
        
        trimmed_data = pd.DataFrame({
            "gamma1": gamma1,
            "gamma0": gamma0,
            "D": trimmed_D.values,
            "Y": trimmed_Y.values,
            "pscore": trimmed_pscore,
        })
        
        # DR Estimate for Y(1)
        trimmed_data['Y1_dr'] = (
            trimmed_data['gamma1'] +
            (trimmed_data['D'] / trimmed_data['pscore']) * (trimmed_data['Y'] - trimmed_data['gamma1'])
        )
        
        # DR Estimate for Y(0)
        trimmed_data['Y0_dr'] = (
            trimmed_data['gamma0'] +
            ((1 - trimmed_data['D']) / (1 - trimmed_data['pscore'])) * (trimmed_data['Y'] - trimmed_data['gamma0'])
        )
        
        # Calculate treatment effect and MSE for the fold
        dr_ate_fold.append(np.mean(trimmed_data['Y1_dr'] - trimmed_data['Y0_dr']))
        mse_fold.append(mean_squared_error(
            trimmed_data['Y'], 
            trimmed_data['Y1_dr'] * trimmed_data['D'] + trimmed_data['Y0_dr'] * (1 - trimmed_data['D'])
        ))
    
    # Average treatment effect and MSE across folds
    dr_ate = np.mean(dr_ate_fold)
    mse = np.mean(mse_fold)
    
    theta_hat_dr_values.append(dr_ate)
    mse_values.append(mse)

# Create results DataFrame
rf_result_df_hetero = pd.DataFrame({
    "Outcome": outcomes,
    "Estimate": theta_hat_dr_values,
    "MSE": mse_values
})

# Print the final results
print("Doubly Robust Treatment Effect Estimates with Cross-Fitting:")
print(rf_result_df_hetero)

Doubly Robust Treatment Effect Estimates with Cross-Fitting:
             Outcome       Estimate           MSE
0           pcorrupt      -0.026246  1.801923e-02
1        ncorrupt_os      -0.007434  2.945456e-03
2      valor_corrupt -144203.110003  8.236523e+11
3  log_valor_corrupt      -0.910680  3.315613e+01


In [101]:
# Checking for balance
weights_treated = 1 / propensity_scores[trimmed_D == 1]
weights_untreated = 1 / (1 - propensity_scores[trimmed_D == 0])

# Create DataFrames for treated and untreated groups
treated_data = trimmed_X[trimmed_D == 1]
untreated_data = trimmed_X[trimmed_D == 0]

# Compute weighted means for each covariate
treated_means = np.average(treated_data, axis=0, weights=weights_treated)
untreated_means = np.average(untreated_data, axis=0, weights=weights_untreated)

# Check absolute standardized mean differences
smd = np.abs(treated_means - untreated_means) / np.std(trimmed_X, axis=0)
print("Standardized Mean Differences (SMD):", smd)

Standardized Mean Differences (SMD): [6.50356092e-02 2.62891458e-01 4.46274547e-02 5.68645557e-01
 1.40846193e-01 2.85167214e-02 2.39017798e-01 4.21856346e-01
 1.70203834e-01 3.12581267e-02 1.11758709e-09 1.11758709e-09
 3.40814162e-01 2.19392841e-01 2.04254184e-01 6.68983541e-04
 1.36899855e-01 1.11758709e-09 1.48401816e-01 1.57523424e-01
 1.77873843e-01 1.11758709e-09 1.64725845e-01 1.52697392e-01
 9.21535698e-03 2.80392638e-01 3.54954060e-01 4.13530648e-01
 1.65639872e-01 5.90287328e-02 1.05886511e-01 7.97249442e-03
 9.18892102e-02 1.52543983e-01 4.31016975e-01 9.91344798e-02
 1.11597675e-01 2.86269278e-01 4.33117663e-01 4.24619704e-01
 2.35066993e-02 1.68955984e-01 1.73020941e-01 1.04110585e-01
 3.15033490e-01 2.44864719e-01 7.19494572e-02 4.65661287e-10
 4.64575176e-01 3.91367717e-01 2.11395745e-02 8.00353031e-03
 2.68994830e-01 1.66763040e-03 1.63270268e-01 1.08851813e-01
 8.79607502e-02 4.67152370e-01 2.46684250e-01 2.45766903e-01
 1.55956523e-01 2.32830644e-10 2.61510988e-02 8.

Balanced

#### Assuming Homogeneous Treatment Effect

In [102]:
theta_hat_dr_values = []
mse_values = []

for outcome in outcomes:
    
    X = double_ml_dataset[all_covariates]
    D = double_ml_dataset[treatment]
    Y = double_ml_dataset[outcome]
    
    # Standardize the covariates
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Prepare 5-Fold Cross-Fitting
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    dr_ate_fold = []
    mse_fold = []
    
    for train_index, test_index in kf.split(X_scaled):
        # Split into training and testing sets
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        D_train, D_test = D.iloc[train_index], D.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
        
        # Step 1: Estimate propensity scores using Random Forest
        rf_pscore = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
        rf_pscore.fit(X_train, D_train)
        propensity_scores = rf_pscore.predict_proba(X_test)[:, 1]
        
        # Step 2: Trim p-scores (0.01, 0.99)
        trimmed_indices = (propensity_scores > 0.01) & (propensity_scores < 0.99)
        trimmed_X = X_test[trimmed_indices]
        trimmed_D = D_test.iloc[trimmed_indices]
        trimmed_Y = Y_test.iloc[trimmed_indices]
        trimmed_pscore = propensity_scores[trimmed_indices]
        
        # Step 3: Fit a single outcome model for homogeneous treatment effect
        outcome_model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
        outcome_model.fit(X_train, Y_train)
        gamma = outcome_model.predict(trimmed_X)
        
        # Step 4: Calculate doubly robust estimates for potential outcomes
        trimmed_data = pd.DataFrame({
            "gamma": gamma,
            "D": trimmed_D.values,
            "Y": trimmed_Y.values,
            "pscore": trimmed_pscore,
        })
        
        # DR Estimate for Y(1)
        trimmed_data['Y1_dr'] = (
            trimmed_data['gamma'] +
            (trimmed_data['D'] / trimmed_data['pscore']) * (trimmed_data['Y'] - trimmed_data['gamma'])
        )
        
        # DR Estimate for Y(0)
        trimmed_data['Y0_dr'] = (
            trimmed_data['gamma'] +
            ((1 - trimmed_data['D']) / (1 - trimmed_data['pscore'])) * (trimmed_data['Y'] - trimmed_data['gamma'])
        )
        
        # Step 5: Calculate treatment effect and MSE for the fold
        dr_ate_fold.append(np.mean(trimmed_data['Y1_dr'] - trimmed_data['Y0_dr']))
        mse_fold.append(mean_squared_error(
            trimmed_data['Y'], 
            trimmed_data['Y1_dr'] * trimmed_data['D'] + trimmed_data['Y0_dr'] * (1 - trimmed_data['D'])
        ))
    
    # Average treatment effect and MSE across folds
    dr_ate = np.mean(dr_ate_fold)
    mse = np.mean(mse_fold)
    
    # Store results
    theta_hat_dr_values.append(dr_ate)
    mse_values.append(mse)

# Step 6: Create results DataFrame
rf_result_df_homo = pd.DataFrame({
    "Outcome": outcomes,
    "Estimate": theta_hat_dr_values,
    "MSE": mse_values
})

# Print the final results
print("Doubly Robust Treatment Effect Estimates with Cross-Fitting (Homogeneous):")
print(rf_result_df_homo)

Doubly Robust Treatment Effect Estimates with Cross-Fitting (Homogeneous):
             Outcome       Estimate           MSE
0           pcorrupt      -0.024637  1.647243e-02
1        ncorrupt_os      -0.007450  2.841831e-03
2      valor_corrupt -130235.474535  6.686565e+11
3  log_valor_corrupt      -0.891584  3.536576e+01


## Neural Network

In [103]:
from sklearn.neural_network import MLPRegressor

In [104]:
# Assuming Heterogeneity

theta_hat_dr_values = []

# Loop through each outcome
for outcome in outcomes:
    
    X = double_ml_dataset[all_covariates]
    D = double_ml_dataset[treatment]
    Y = double_ml_dataset[outcome]
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Prepare 5-Fold Cross-Fitting
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    dr_ate_fold = []
    
    for train_index, test_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        D_train, D_test = D.iloc[train_index], D.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
        
        # Estimate propensity scores using Random Forest
        rf_pscore = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
        rf_pscore.fit(X_train, D_train)
        propensity_scores = rf_pscore.predict_proba(X_test)[:, 1]  # Probability of treatment
        
        # Trim p-scores (0.01, 0.99)
        trimmed_indices = (propensity_scores > 0.01) & (propensity_scores < 0.99)
        trimmed_X = X_test[trimmed_indices]
        trimmed_D = D_test.iloc[trimmed_indices]
        trimmed_Y = Y_test.iloc[trimmed_indices]
        trimmed_pscore = propensity_scores[trimmed_indices]
        
        # Fit outcome models for treated (D=1) and untreated (D=0) groups using Random Forest
        # Treated model
        treated_model = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42)
        treated_model.fit(X_train[D_train == 1], Y_train[D_train == 1])
        gamma1 = treated_model.predict(trimmed_X)
        
        # Untreated model
        untreated_model = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42)
        untreated_model.fit(X_train[D_train == 0], Y_train[D_train == 0])
        gamma0 = untreated_model.predict(trimmed_X)
        
        trimmed_data = pd.DataFrame({
            "gamma1": gamma1,
            "gamma0": gamma0,
            "D": trimmed_D.values,
            "Y": trimmed_Y.values,
            "pscore": trimmed_pscore,
        })
        
        # DR Estimate for Y(1)
        trimmed_data['Y1_dr'] = (
            trimmed_data['gamma1'] +
            (trimmed_data['D'] / trimmed_data['pscore']) * (trimmed_data['Y'] - trimmed_data['gamma1'])
        )
        
        # DR Estimate for Y(0)
        trimmed_data['Y0_dr'] = (
            trimmed_data['gamma0'] +
            ((1 - trimmed_data['D']) / (1 - trimmed_data['pscore'])) * (trimmed_data['Y'] - trimmed_data['gamma0'])
        )
        
        dr_ate_fold.append(np.mean(trimmed_data['Y1_dr'] - trimmed_data['Y0_dr']))
    
    # Average treatment effect across folds
    dr_ate = np.mean(dr_ate_fold)
    theta_hat_dr_values.append(dr_ate)

# Print the final treatment effect estimates
print("Doubly Robust Treatment Effect Estimates with Cross-Fitting:")
print(theta_hat_dr_values)

Doubly Robust Treatment Effect Estimates with Cross-Fitting:
[np.float64(-0.04054703756589591), np.float64(-0.024202559561389708), np.float64(-129347.09336311887), np.float64(-1.346402764310549)]


## Gradient Boosting


In [105]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

#### Assuming Heteogeneous treatment effct 

In [109]:
theta_hat_dr_values = []
mse_values = []

# Loop through each outcome
for outcome in outcomes:
    
    X = double_ml_dataset[all_covariates]
    D = double_ml_dataset[treatment]
    Y = double_ml_dataset[outcome]
    
    # Standardize the covariates
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Prepare 5-Fold Cross-Fitting
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    dr_ate_fold = []
    mse_fold = []
    
    for train_index, test_index in kf.split(X_scaled):
        # Split into training and testing sets
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        D_train, D_test = D.iloc[train_index], D.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
        
        # Step 1: Estimate propensity scores using Gradient Boosting
        gb_pscore = GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=42)
        gb_pscore.fit(X_train, D_train)
        propensity_scores = gb_pscore.predict_proba(X_test)[:, 1]  # Probability of treatment
        
        # Step 2: Trim p-scores (0.01, 0.99)
        trimmed_indices = (propensity_scores > 0.01) & (propensity_scores < 0.99)
        trimmed_X = X_test[trimmed_indices]
        trimmed_D = D_test.iloc[trimmed_indices]
        trimmed_Y = Y_test.iloc[trimmed_indices]
        trimmed_pscore = propensity_scores[trimmed_indices]
        
        # Step 3: Fit outcome models for treated (D=1) and untreated (D=0) groups using Gradient Boosting
        treated_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
        treated_model.fit(X_train[D_train == 1], Y_train[D_train == 1])
        gamma1 = treated_model.predict(trimmed_X)
        
        untreated_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
        untreated_model.fit(X_train[D_train == 0], Y_train[D_train == 0])
        gamma0 = untreated_model.predict(trimmed_X)
        
        # Step 4: Calculate doubly robust estimates for potential outcomes
        trimmed_data = pd.DataFrame({
            "gamma1": gamma1,
            "gamma0": gamma0,
            "D": trimmed_D.values,
            "Y": trimmed_Y.values,
            "pscore": trimmed_pscore,
        })
        
        # DR Estimate for Y(1)
        trimmed_data['Y1_dr'] = (
            trimmed_data['gamma1'] +
            (trimmed_data['D'] / trimmed_data['pscore']) * (trimmed_data['Y'] - trimmed_data['gamma1'])
        )
        
        # DR Estimate for Y(0)
        trimmed_data['Y0_dr'] = (
            trimmed_data['gamma0'] +
            ((1 - trimmed_data['D']) / (1 - trimmed_data['pscore'])) * (trimmed_data['Y'] - trimmed_data['gamma0'])
        )
        
        # Step 5: Calculate treatment effect and MSE for the fold
        dr_ate_fold.append(np.mean(trimmed_data['Y1_dr'] - trimmed_data['Y0_dr']))
        mse_fold.append(mean_squared_error(
            trimmed_data['Y'], 
            trimmed_data['Y1_dr'] * trimmed_data['D'] + trimmed_data['Y0_dr'] * (1 - trimmed_data['D'])
        ))
    
    # Average treatment effect and MSE across folds
    dr_ate = np.mean(dr_ate_fold)
    mse = np.mean(mse_fold)
    
    theta_hat_dr_values.append(dr_ate)
    mse_values.append(mse)

# Step 6: Create results DataFrame
gb_result_df_hetero = pd.DataFrame({
    "Outcome": outcomes,
    "Estimate": theta_hat_dr_values,
    "MSE": mse_values
})

# Print the final results
print("Doubly Robust Treatment Effect Estimates with Cross-Fitting (Gradient Boosting):")
print(gb_result_df_hetero)

Doubly Robust Treatment Effect Estimates with Cross-Fitting (Gradient Boosting):
             Outcome       Estimate           MSE
0           pcorrupt      -0.025620  1.416204e-01
1        ncorrupt_os      -0.006851  2.309279e-02
2      valor_corrupt -100519.402595  3.362692e+12
3  log_valor_corrupt      -0.247871  3.027244e+02


In [110]:
# Checking for balance
weights_treated = 1 / propensity_scores[trimmed_D == 1]
weights_untreated = 1 / (1 - propensity_scores[trimmed_D == 0])

# Create DataFrames for treated and untreated groups
treated_data = trimmed_X[trimmed_D == 1]
untreated_data = trimmed_X[trimmed_D == 0]

# Compute weighted means for each covariate
treated_means = np.average(treated_data, axis=0, weights=weights_treated)
untreated_means = np.average(untreated_data, axis=0, weights=weights_untreated)

# Check absolute standardized mean differences
smd = np.abs(treated_means - untreated_means) / np.std(trimmed_X, axis=0)
print("Standardized Mean Differences (SMD):", smd)

Standardized Mean Differences (SMD): [1.30354360e-01 4.08693582e-01 1.33388510e-01 4.50837250e-01
 2.72433543e-01 5.42745332e-02 2.11132202e-01 5.82719950e-01
 5.05626023e-02 1.76033370e-01 1.11758709e-09 1.11758709e-09
 3.30145114e-01 1.48642464e-01 1.99095532e-01 1.62103395e-01
 1.13239356e-01 3.72529030e-10 1.03624717e-01 1.12928497e-01
 2.87994355e-01 1.11758709e-09 2.36576992e-01 4.62915109e-01
 2.39045559e-01 3.70670252e-01 1.91615557e-01 4.10861657e-01
 8.92049971e-02 9.31998437e-03 2.48399569e-02 7.81668308e-04
 9.20056348e-02 1.98191258e-01 5.65954375e-01 1.34008938e-01
 6.06387170e-02 2.07655034e-01 3.35775610e-01 6.40965187e-01
 1.16275995e-02 1.94303309e-01 1.26887087e-01 1.86793530e-01
 2.58440935e-01 2.53349413e-01 2.24380029e-01 9.31322575e-10
 5.89912083e-01 3.46950845e-01 3.48494447e-02 4.84613145e-02
 2.28529361e-01 1.73389753e-02 2.19947856e-01 9.38967025e-03
 2.32028895e-01 5.85672262e-01 5.60779213e-02 1.57979067e-01
 9.63840800e-02 6.98491931e-10 3.56220886e-01 6.

Balanced

#### Assuming Homogeneous treatment effct 

In [108]:
theta_hat_dr_values = []
mse_values = []

# Loop through each outcome
for outcome in outcomes:

    X = double_ml_dataset[all_covariates]
    D = double_ml_dataset[treatment]
    Y = double_ml_dataset[outcome]
    
    # Standardize the covariates
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Prepare 5-Fold Cross-Fitting
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    dr_ate_fold = []
    mse_fold = []
    
    for train_index, test_index in kf.split(X_scaled):
        # Split into training and testing sets
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        D_train, D_test = D.iloc[train_index], D.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
        
        # Step 1: Estimate propensity scores using Gradient Boosting
        gb_pscore = GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=42)
        gb_pscore.fit(X_train, D_train)
        propensity_scores = gb_pscore.predict_proba(X_test)[:, 1]
        
        # Step 2: Trim p-scores (0.01, 0.99)
        trimmed_indices = (propensity_scores > 0.01) & (propensity_scores < 0.99)
        trimmed_X = X_test[trimmed_indices]
        trimmed_D = D_test.iloc[trimmed_indices]
        trimmed_Y = Y_test.iloc[trimmed_indices]
        trimmed_pscore = propensity_scores[trimmed_indices]
        
        # Step 3: Fit a single outcome model for homogeneous treatment effect using Gradient Boosting
        outcome_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
        outcome_model.fit(X_train, Y_train)
        gamma = outcome_model.predict(trimmed_X)
        
        # Step 4: Calculate doubly robust estimates for potential outcomes
        trimmed_data = pd.DataFrame({
            "gamma": gamma,
            "D": trimmed_D.values,
            "Y": trimmed_Y.values,
            "pscore": trimmed_pscore,
        })
        
        # DR Estimate for Y(1)
        trimmed_data['Y1_dr'] = (
            trimmed_data['gamma'] +
            (trimmed_data['D'] / trimmed_data['pscore']) * (trimmed_data['Y'] - trimmed_data['gamma'])
        )
        
        # DR Estimate for Y(0)
        trimmed_data['Y0_dr'] = (
            trimmed_data['gamma'] +
            ((1 - trimmed_data['D']) / (1 - trimmed_data['pscore'])) * (trimmed_data['Y'] - trimmed_data['gamma'])
        )
        
        # Step 5: Calculate treatment effect and MSE for the fold
        dr_ate_fold.append(np.mean(trimmed_data['Y1_dr'] - trimmed_data['Y0_dr']))
        mse_fold.append(mean_squared_error(
            trimmed_data['Y'], 
            trimmed_data['Y1_dr'] * trimmed_data['D'] + trimmed_data['Y0_dr'] * (1 - trimmed_data['D'])
        ))
    
    # Average treatment effect and MSE across folds
    dr_ate = np.mean(dr_ate_fold)
    mse = np.mean(mse_fold)
    
    # Store results
    theta_hat_dr_values.append(dr_ate)
    mse_values.append(mse)

# Step 6: Create results DataFrame
gb_result_df_homo = pd.DataFrame({
    "Outcome": outcomes,
    "Estimate": theta_hat_dr_values,
    "MSE": mse_values
})

# Print the final results
print("Doubly Robust Treatment Effect Estimates with Cross-Fitting (Homogeneous, Gradient Boosting):")
print(gb_result_df_homo)

Doubly Robust Treatment Effect Estimates with Cross-Fitting (Homogeneous, Gradient Boosting):
             Outcome       Estimate           MSE
0           pcorrupt      -0.020634  2.081520e-01
1        ncorrupt_os      -0.008694  1.700983e-02
2      valor_corrupt -204342.791258  2.918803e+12
3  log_valor_corrupt      -0.947817  2.592390e+02


## Analysis 

compare mse/theta,


## Conclusion